In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import sys
sys.path.insert(0, "../../")
from topological_codes import RotatedDecoder
from topological_codes import XXZZQubit

In [25]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit import execute, Aer

def get_noise_model(p_err):

    error_gate1 = pauli_error([("X", p_err / 2), ("Z", p_err / 2), ("I", 1 - p_err)])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, "id")
    return noise_model

In [26]:
p_err = 0.10
shots = 10000
def get_results(circ):
    results = execute(circ, 
                      Aer.get_backend("aer_simulator"), 
                      noise_model=get_noise_model(p_err),
                      shots=shots,
                     ).result().get_counts()
    return results

In [27]:
d = 3
T = 1

decoder = RotatedDecoder({"d":d,"T":T})

In [29]:
qubit = XXZZQubit({'d':d})
qubit.logical_plus_z_reset()
qubit.stabilize()
qubit.identity_data()
qubit.stabilize()
qubit.readout_z()
decoder.set_look_up_table("Z",0,get_results(qubit.circ))

qubit = XXZZQubit({'d':d})
qubit.logical_plus_z_reset()
qubit.logical_x()
qubit.stabilize()
qubit.identity_data()
qubit.stabilize()
qubit.readout_z()
decoder.set_look_up_table("Z",1,get_results(qubit.circ))

In [30]:
readout_string = "1 00000000 00000000"
readout_type = "Z"
decoder.correct_readout_look_up_table(readout_string, readout_type)

1